# 线性回归的简洁实现

通过使用深度学习框架来简洁地实现
线性回归模型
生成数据集

In [13]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)
print('features:', features.shape, '\nlabel:', labels.shape)

features: torch.Size([1000, 2]) 
label: torch.Size([1000, 1])


调用框架中现有的API来读取数据

In [14]:
'''
Author       : JonnyZhang 71881972+jonnyzhang02@users.noreply.github.com
LastEditTime : 2023-07-18 18:51
FilePath     : \d2l-zh-pytorch\chapter_linear-networks\linear-regression-concise.ipynb

coded by ZhangYang@BUPT, my email is zhangynag0207@bupt.edu.cn
'''
def load_array(data_arrays, batch_size, is_train=True):  
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    print(dataset[0])
    # shuffle: 是否需要打乱数据
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

next(iter(data_iter))

(tensor([-0.7544,  0.0232]), tensor([2.6156]))


[tensor([[ 1.1843, -0.5124],
         [-0.1763,  0.4498],
         [-1.5459,  1.7881],
         [-1.1444,  0.5178],
         [-1.4904, -0.0096],
         [-0.6372, -0.5577],
         [ 0.0734, -0.5525],
         [ 0.7379,  1.7341],
         [-0.1173,  0.9498],
         [ 0.0948, -1.6577]]),
 tensor([[ 8.3007],
         [ 2.3187],
         [-4.9619],
         [ 0.1427],
         [ 1.2493],
         [ 4.8136],
         [ 6.2474],
         [-0.2218],
         [ 0.7421],
         [10.0243]])]

使用框架的预定义好的层

In [15]:
from torch import nn

# `nn` 是神经网络的缩写
# Sequential实例可以看作是一个串联各个层的容器
# 线性回归就是一个单层线性神经网络
net = nn.Sequential(nn.Linear(2, 1))

初始化模型参数

In [16]:
net[0].weight.data.normal_(0, 0.01)  # 初始化权重
net[0].bias.data.fill_(0)            # 初始化偏差

tensor([0.])

计算均方误差使用的是`MSELoss`类，也称为平方$L_2$范数

In [17]:
loss = nn.MSELoss()   # 均方损失函数
# MSE : Mean Squared Error  pytorch已经求平均了

实例化一个`SGD`实例

In [18]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)
# SGD: 随机梯度下降 (Stochastic Gradient Descent)

训练过程代码与我们从零开始实现时所做的非常相似

In [19]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad() # 梯度清零，等价于net.zero_grad()
        l.backward()        # 反向传播
        trainer.step()      # 更新参数
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000327
epoch 2, loss 0.000109
epoch 3, loss 0.000110


比较生成数据集的真实参数和通过有限数据训练获得的模型参数

In [20]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([-0.0012, -0.0005])
b的估计误差： tensor([-0.0002])
